In [1]:
import numpy as np
import polars as pl
import time
from sklearn import tree
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from google.colab import drive                       #This is to import the datasets which are uploaded to Google drive
drive.mount('/content/drive')
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import TargetEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

Mounted at /content/drive


In [2]:
Df = pl.read_csv('/content/drive/MyDrive/Datasets/US Accidents/US_Accidents_March23.csv',n_rows = 1000500)

In [3]:
Df.head(5)

ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,i64,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str
"""A-1""","""Source2""",3,"""2016-02-08 05:46:00""","""2016-02-08 11:00:00""",39.865147,-84.058723,null,null,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US""","""US/Eastern""","""KFFO""","""2016-02-08 05:58:00""",36.9,null,91.0,29.68,10.0,"""Calm""",null,0.02,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Night"""
"""A-2""","""Source2""",2,"""2016-02-08 06:07:59""","""2016-02-08 06:37:59""",39.928059,-82.831184,null,null,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US""","""US/Eastern""","""KCMH""","""2016-02-08 05:51:00""",37.9,null,100.0,29.65,10.0,"""Calm""",null,0.0,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Day"""
"""A-3""","""Source2""",2,"""2016-02-08 06:49:27""","""2016-02-08 07:19:27""",39.063148,-84.032608,null,null,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""","""2016-02-08 06:56:00""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,null,"""Overcast""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""",3,"""2016-02-08 07:23:34""","""2016-02-08 07:53:34""",39.747753,-84.205582,null,null,0.01,"""Accident on I-75 Southbound at…","""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""","""2016-02-08 07:38:00""",35.1,31.0,96.0,29.64,9.0,"""SW""",4.6,null,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Day""","""Day""","""Day"""
"""A-5""","""Source2""",2,"""2016-02-08 07:39:07""","""2016-02-08 08:09:07""",39.627781,-84.188354,null,null,0.01,"""Accident on McEwen Rd at OH-72…","""Miamisburg Centerville Rd""","""Dayton""","""Montgomery""","""OH""","""45459""","""US""","""US/Eastern""","""KMGY""","""2016-02-08 07:53:00""",36.0,33.3,89.0,29.65,6.0,"""SW""",3.5,null,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Day""","""Day""","""Day""","""Day"""


In [4]:
Df.describe()

statistic,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,str,f64,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str
"""count""","""1000500""","""1000500""",1.0005e6,"""1000500""","""1000500""",1.0005e6,1.0005e6,"""0""","""0""",1.0005e6,"""1000499""","""998788""","""1000472""","""1000500""","""1000500""","""1000364""","""1000500""","""1000047""","""999066""","""990569""",985192.0,539898.0,983937.0,988118.0,981562.0,"""986920""",900092.0,525177.0,"""982282""",1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,1.0005e6,"""998864""","""998864""","""998864""","""998864"""
"""null_count""","""0""","""0""",0.0,"""0""","""0""",0.0,0.0,"""1000500""","""1000500""",0.0,"""1""","""1712""","""28""","""0""","""0""","""136""","""0""","""453""","""1434""","""9931""",15308.0,460602.0,16563.0,12382.0,18938.0,"""13580""",100408.0,475323.0,"""18218""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""1636""","""1636""","""1636""","""1636"""
"""mean""",null,null,2.320224,null,null,36.058837,-94.750544,null,null,0.167532,null,null,null,null,null,null,null,null,null,null,64.626661,59.457691,66.029973,29.693456,9.173896,null,7.767402,0.013247,null,0.013918,0.000321,0.129516,0.006232,0.056812,0.002171,0.00933,0.000027,0.027968,0.029207,0.000887,0.209889,0.0,null,null,null,null
"""std""",null,null,0.548461,null,null,4.906894,17.388672,null,null,1.427759,null,null,null,null,null,null,null,null,null,null,17.450303,21.930264,21.888529,0.792037,2.259134,null,5.407576,0.203059,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""A-1""","""Source2""",1.0,"""2016-02-08 05:46:00""","""2016-02-08 06:37:59""",24.5548,-124.497414,null,null,0.0,""" middle lane blocked due to ac…","""00-199 Fair Lawn Pkwy""","""Aaronsburg""","""Abbeville""","""AL""","""01001""","""US""","""US/Central""","""K01M""","""2016-02-08 05:51:00""",-77.8,-50.0,1.0,0.12,0.0,"""CALM""",0.0,0.0,"""Blowing Dust""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Day""","""Day""","""Day""","""Day"""
"""25%""",null,null,2.0,null,null,32.925362,-117.36837,null,null,0.0,null,null,null,null,null,null,null,null,null,null,54.0,43.0,50.0,29.56,10.0,null,4.6,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,null,2.0,null,null,35.257641,-87.957573,null,null,0.0,null,null,null,null,null,null,null,null,null,null,66.9,64.0,68.0,29.91,10.0,null,7.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,3.0,null,null,40.11311,-80.920403,null,null,0.01,null,null,null,null,null,null,null,null,null,null,77.0,76.0,84.0,30.05,10.0,null,10.4,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""max""","""A-999999""","""Source3""",4.0,"""2022-09-08 17:56:22""","""2022-09-08 18:34:31""",48.998085,-68.160789,null,null,336.570007,"""~Left lane blocked due to cras…","""william Carey Dr""","""Zumbrota""","""Yuma""","""WY""","""99362-1879""","""US""","""US/Pacific""","""KZZV""","""2022-09-08 17:55:00""",196.0,196.0,100.0,58.1,111.0,"""West""",822.8,36.47,"""Wintry Mix / Windy""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,"""Night""","""Night""","""Night""","""Night"""


## Steps to control leakage:

In [5]:
#Dropping columns to reduce leakage
Df = Df.drop(['End_Time','End_Lat','End_Lng','Description','Weather_Timestamp'])

In [6]:
Df.head(5)

ID,Source,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,i64,str,f64,f64,f64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str
"""A-1""","""Source2""",3,"""2016-02-08 05:46:00""",39.865147,-84.058723,0.01,"""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US""","""US/Eastern""","""KFFO""",36.9,null,91.0,29.68,10.0,"""Calm""",null,0.02,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Night"""
"""A-2""","""Source2""",2,"""2016-02-08 06:07:59""",39.928059,-82.831184,0.01,"""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US""","""US/Eastern""","""KCMH""",37.9,null,100.0,29.65,10.0,"""Calm""",null,0.0,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Day"""
"""A-3""","""Source2""",2,"""2016-02-08 06:49:27""",39.063148,-84.032608,0.01,"""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,null,"""Overcast""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""",3,"""2016-02-08 07:23:34""",39.747753,-84.205582,0.01,"""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""",35.1,31.0,96.0,29.64,9.0,"""SW""",4.6,null,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Day""","""Day""","""Day"""
"""A-5""","""Source2""",2,"""2016-02-08 07:39:07""",39.627781,-84.188354,0.01,"""Miamisburg Centerville Rd""","""Dayton""","""Montgomery""","""OH""","""45459""","""US""","""US/Eastern""","""KMGY""",36.0,33.3,89.0,29.65,6.0,"""SW""",3.5,null,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Day""","""Day""","""Day""","""Day"""


# Handling NULL values and EDA:

In [7]:
Df.is_duplicated()[True]

False

There are no duplicate rows.

In [8]:
Df.select(pl.all().is_null().sum())

ID,Source,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,1712,28,0,0,136,0,453,1434,15308,460602,16563,12382,18938,13580,100408,475323,18218,0,0,0,0,0,0,0,0,0,0,0,0,0,1636,1636,1636,1636


In [9]:
#Imputing null values in numerical features with last valid value in the particular column
Df = Df.with_columns(pl.col("Temperature(F)").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Wind_Chill(F)").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Humidity(%)").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Pressure(in)").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Visibility(mi)").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Wind_Speed(mph)").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Precipitation(in)").fill_null(strategy="forward"))

In [10]:
Df.select(pl.all().is_null().sum())

ID,Source,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,1712,28,0,0,136,0,453,1434,0,2,0,0,0,13580,2,0,18218,0,0,0,0,0,0,0,0,0,0,0,0,0,1636,1636,1636,1636


In [11]:
Df.head(2)

ID,Source,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,i64,str,f64,f64,f64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str
"""A-1""","""Source2""",3,"""2016-02-08 05:46:00""",39.865147,-84.058723,0.01,"""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US""","""US/Eastern""","""KFFO""",36.9,null,91.0,29.68,10.0,"""Calm""",null,0.02,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Night"""
"""A-2""","""Source2""",2,"""2016-02-08 06:07:59""",39.928059,-82.831184,0.01,"""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US""","""US/Eastern""","""KCMH""",37.9,null,100.0,29.65,10.0,"""Calm""",null,0.0,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Day"""


In [12]:
#Imputing string features with high null value count with forward strategy imputation:
Df = Df.with_columns(pl.col("Wind_Direction").fill_null(strategy="forward"))
Df = Df.with_columns(pl.col("Weather_Condition").fill_null(strategy="forward"))

In [13]:
#Dropping all remaining rows containing null values as they are lesser in number w.r.t. the size of the total dataset:
Df = Df.drop_nulls(subset = ["Street","City","Zipcode","Timezone","Airport_Code","Wind_Chill(F)","Wind_Speed(mph)","Sunrise_Sunset","Civil_Twilight","Nautical_Twilight","Astronomical_Twilight"])

In [14]:
Df.select(pl.all().is_null().sum())

ID,Source,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


All null values have been taken care of.

In [15]:
#Removing the target variable before scaling
target_series = Df['Severity']
Df = Df.drop(['Severity'])

# Scaling Numerical features:

In [16]:
import polars.selectors as cs
from sklearn.preprocessing import StandardScaler

In [17]:
Df = Df.with_columns([
    ((pl.col(x) - pl.col(x).mean()) / pl.col(x).std()).alias(x)
    for x in Df.select(cs.numeric()).columns
])

In [18]:
Df

ID,Source,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str
"""A-3""","""Source2""","""2016-02-08 06:49:27""",0.611725,0.618225,-0.110022,"""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""",-1.640993,-1.211852,1.550824,-0.030468,0.366151,"""SW""",-0.761427,-0.083077,"""Overcast""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""","""2016-02-08 07:23:34""",0.751197,0.608281,-0.110022,"""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""",-1.692592,-1.323379,1.368134,-0.068358,-0.076316,"""SW""",-0.557635,-0.083077,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Day""","""Day""","""Day"""
"""A-5""","""Source2""","""2016-02-08 07:39:07""",0.726755,0.609271,-0.110022,"""Miamisburg Centerville Rd""","""Dayton""","""Montgomery""","""OH""","""45459""","""US""","""US/Eastern""","""KMGY""",-1.640993,-1.211852,1.048427,-0.055728,-1.403715,"""SW""",-0.761427,-0.083077,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Day""","""Day""","""Day""","""Day"""
"""A-6""","""Source2""","""2016-02-08 07:44:26""",0.823079,0.681893,-0.110022,"""Westerville Rd""","""Westerville""","""Franklin""","""OH""","""43081""","""US""","""US/Eastern""","""KCMH""",-1.532061,-1.105173,1.413806,-0.080988,-0.961249,"""SSW""",-0.761427,-0.037278,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Day""","""Day""","""Day""","""Day"""
"""A-7""","""Source2""","""2016-02-08 07:59:35""",0.75334,0.606848,-0.117097,"""N Woodward Ave""","""Dayton""","""Montgomery""","""OH""","""45417-2476""","""US""","""US/Eastern""","""KDAY""",-1.755658,-1.323379,1.550824,-0.043098,-0.961249,"""WSW""",-0.761427,-0.037278,"""Overcast""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Day""","""Day""","""Day""","""Day"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A-1010261""","""Source2""","""2021-06-10 11:25:11""",-0.643856,-0.111083,-0.117097,"""Skillman St""","""Dallas""","""Dallas""","""TX""","""75243""","""US""","""US/Central""","""KADS""",1.110973,1.246596,0.180651,-0.5483,0.366151,"""S""",0.257531,-0.083077,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Day""","""Day""","""Day""","""Day"""
"""A-1010262""","""Source2""","""2021-06-10 12:10:55""",-0.666035,-0.112231,-0.117097,"""I-30 E""","""Dallas""","""Dallas""","""TX""","""75223""","""US""","""US/Central""","""KDAL""",1.05364,1.198106,0.134979,-0.346219,0.366151,"""SSE""",1.183857,-0.083077,"""Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Day""","""Day""","""Day""","""Day"""
"""A-1010263""","""Source2""","""2021-06-10 12:19:56""",-0.651048,-0.101044,-0.117097,"""E Interstate 30""","""Garland""","""Dallas""","""TX""","""75043-4453""","""US""","""US/Central""","""KHQZ""",1.110973,1.246596,0.180651,-0.257809,0.366151,"""S""",0.813327,-0.083077,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Day""","""Day""","""Day""","""Day"""


# Encoding Boolean features:

In [19]:
Df.head(2)

ID,Source,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str
"""A-3""","""Source2""","""2016-02-08 06:49:27""",0.611725,0.618225,-0.110022,"""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""",-1.640993,-1.211852,1.550824,-0.030468,0.366151,"""SW""",-0.761427,-0.083077,"""Overcast""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""","""2016-02-08 07:23:34""",0.751197,0.608281,-0.110022,"""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""",-1.692592,-1.323379,1.368134,-0.068358,-0.076316,"""SW""",-0.557635,-0.083077,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Day""","""Day""","""Day"""


In [20]:
Df = Df.with_columns([
    pl.col("Amenity").cast(pl.Float32),
    pl.col("Bump").cast(pl.Float32),
    pl.col("Crossing").cast(pl.Float32),
    pl.col("Give_Way").cast(pl.Float32),
    pl.col("Junction").cast(pl.Float32),
    pl.col("No_Exit").cast(pl.Float32),
    pl.col("Railway").cast(pl.Float32),
    pl.col("Roundabout").cast(pl.Float32),
    pl.col("Station").cast(pl.Float32),
    pl.col("Stop").cast(pl.Float32),
    pl.col("Traffic_Calming").cast(pl.Float32),
    pl.col("Traffic_Signal").cast(pl.Float32),
    pl.col("Turning_Loop").cast(pl.Float32)
])

In [21]:
Df.head(2)

ID,Source,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,str,str
"""A-3""","""Source2""","""2016-02-08 06:49:27""",0.611725,0.618225,-0.110022,"""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""",-1.640993,-1.211852,1.550824,-0.030468,0.366151,"""SW""",-0.761427,-0.083077,"""Overcast""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""","""2016-02-08 07:23:34""",0.751197,0.608281,-0.110022,"""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""",-1.692592,-1.323379,1.368134,-0.068358,-0.076316,"""SW""",-0.557635,-0.083077,"""Mostly Cloudy""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Night""","""Day""","""Day""","""Day"""


# Casting all non-categorical features to float32:

In [22]:
Df.head(2)

ID,Source,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,str,str
"""A-3""","""Source2""","""2016-02-08 06:49:27""",0.611725,0.618225,-0.110022,"""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""",-1.640993,-1.211852,1.550824,-0.030468,0.366151,"""SW""",-0.761427,-0.083077,"""Overcast""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""","""2016-02-08 07:23:34""",0.751197,0.608281,-0.110022,"""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""",-1.692592,-1.323379,1.368134,-0.068358,-0.076316,"""SW""",-0.557635,-0.083077,"""Mostly Cloudy""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Night""","""Day""","""Day""","""Day"""


In [29]:
Df = Df.with_columns([
    (pl.col("Start_Lat").cast(pl.Float32)),
    (pl.col("Start_Lng").cast(pl.Float32)),
    (pl.col("Distance(mi)").cast(pl.Float32)),
    (pl.col("Temperature(F)").cast(pl.Float32)),
    (pl.col("Wind_Chill(F)").cast(pl.Float32)),
    (pl.col("Humidity(%)").cast(pl.Float32)),
    (pl.col("Pressure(in)").cast(pl.Float32)),
    (pl.col("Visibility(mi)").cast(pl.Float32)),
    (pl.col("Precipitation(in)").cast(pl.Float32)),
]
)

In [30]:
Df.head(2)

ID,Source,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,str,f32,f32,f32,str,str,str,str,str,str,str,str,f32,f32,f32,f32,f32,str,f64,f32,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,str,str
"""A-3""","""Source2""","""2016-02-08 06:49:27""",0.611725,0.618225,-0.110022,"""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""",-1.640993,-1.211852,1.550824,-0.030468,0.366151,"""SW""",-0.761427,-0.083077,"""Overcast""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""","""2016-02-08 07:23:34""",0.751197,0.608281,-0.110022,"""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""",-1.692592,-1.323379,1.368134,-0.068358,-0.076316,"""SW""",-0.557635,-0.083077,"""Mostly Cloudy""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Night""","""Day""","""Day""","""Day"""


In [31]:
target_series

Severity
i64
2
3
2
3
2
…
3
3
3


# Splitting training and testing data: